In [100]:
%load_ext autoreload
%autoreload 2
import time
from klampt import *
from klampt.math import vectorops,so3,se3
from klampt.io import loader
import math
import random
from IPython.display import clear_output,Markdown
import ipywidgets as widgets
import ipyklampt

world = WorldModel()
fn = "data/ur5Blocks.xml"
res = world.readFile(fn)
robot = world.robot(0)

# perturb the block configuration
objectTransforms = [world.rigidObject(i).getTransform() for i in range(world.numRigidObjects())]
random.shuffle(objectTransforms)
for i in range(world.numRigidObjects()):
    R,t = objectTransforms[i]
    t[0] += random.uniform(-0.05,0.05)
    t[1] += random.uniform(-0.05,0.05)
    R = so3.from_axis_angle(((0,0,1),random.uniform(0,math.pi*2)))
    world.rigidObject(i).setTransform(R,t)

#set the home configuration
qhome  = robot.getConfig()
qhome[1] = 2.3562
qhome[2] = -math.pi/3.0
qhome[3] = math.pi*2/3.0
qhome[4] = 0.64181
qhome[5] = math.pi/2.0

kvis = ipyklampt.KlamptWidget(world,width=600,height=400)
# kvis.set_camera({u'position': {u'y': 1.9298983866544108, u'x': 0.7498269220525384, u'z': 1.3093949571767591}, u'up': {u'y': 0.9488807720023056, u'x': 0.06236278376382235, u'z': -0.30941261080559734}, u'target': {u'y': 0.6599169321160283, u'x': -0.12092171623712519, u'z': 0.08376713045866419}})


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
#set up the controller and useful subroutines for converting to/from Klampt
def controller_to_klampt(qcontroller):
    """Converts a 6+1 controller configuration to Klampt format"""
    qorig = robot.getConfig()
    q = [v for v in qorig]
    for i in range(6):
        q[i+1] = qcontroller[i]
    robot.setConfig(q)
    robot.driver(6).setValue(qcontroller[6])
    q = robot.getConfig()
    robot.setConfig(qorig)
    return q

def klampt_to_controller(qklampt):
    """Converts a Klamp't configuration to 6+1 controller format"""
    qorig = robot.getConfig()
    res = [qklampt[i+1] for i in range(6)]
    robot.setConfig(qklampt)
    res.append(robot.driver(6).getValue())
    robot.setConfig(qorig)
    return res

CONTROLLER = 'simulation'
#when you are ready, replace the above line with this
#CONTROLLER = 'physical'
if CONTROLLER == 'simulation':
    from RobotControllerEmulator import kill_controller_threads,UR5WithGripperController
    controllerWorld = world.copy()
    kvis_ground_truth = ipyklampt.KlamptWidget(controllerWorld,width=600,height=400)
    robot_control_api = UR5WithGripperController(controllerWorld,randomize=True)
    VIS_UPDATE_FREQUENCY = 10
    VIS_UPDATE_ITERS = 0
    def _update_vis_hook():
        global VIS_UPDATE_ITERS
        VIS_UPDATE_ITERS += 1
        if VIS_UPDATE_ITERS % VIS_UPDATE_FREQUENCY == 0:
            #this is called inside the thread
            kvis_ground_truth.update()
    def start_controller():
        kill_controller_threads()
        robot_control_api.start(_update_vis_hook)
    #kvis_ground_truth.set_camera({u'position': {u'y': 1.9298983866544108, u'x': 0.7498269220525384, u'z': 1.3093949571767591}, u'up': {u'y': 0.9488807720023056, u'x': 0.06236278376382235, u'z': -0.30941261080559734}, u'target': {u'y': 0.6599169321160283, u'x': -0.12092171623712519, u'z': 0.08376713045866419}})
else:
    from robot_api.RobotController import UR5WithGripperController
    robot_control_api = UR5WithGripperController(host='192.168.0.1')
    def start_controller():
        robot_control_api.start()
def update_vis_from_controller():
    q = robot_control_api.getConfig()
    robot.setConfig(controller_to_klampt(q))
    kvis.update()
def stop_controller():
    robot_control_api.stop()
def wait_controller(duration):
    t0 = time.time()
    tstart = t0
    while time.time()-tstart < duration:
        update_vis_from_controller()
        t1 = time.time()
        time.sleep(max(0,0.05-(t1-t0)))
        t0 = t1


In [102]:
if CONTROLLER == 'simulation':
    display(Markdown("## Ground truth world"))
    display(kvis_ground_truth)

controls = widgets.HBox([])
display(controls)

display(Markdown("## Planning world"))
display(kvis)

output_area = widgets.Output()
display(output_area)

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

## Ground truth world

S2xhbXB0V2lkZ2V0KGhlaWdodD00MDAsIHNjZW5lPXt1J29iamVjdCc6IHt1J21hdHJpeCc6IFsxLCAwLCAwLCAwLCAwLCAwLCAtMSwgMCwgMCwgMSwgMCwgMCwgMCwgMCwgMCwgMV0sIHUndXXigKY=


HBox()

## Planning world

S2xhbXB0V2lkZ2V0KGhlaWdodD00MDAsIHNjZW5lPXt1J29iamVjdCc6IHt1J21hdHJpeCc6IFsxLCAwLCAwLCAwLCAwLCAwLCAtMSwgMCwgMCwgMSwgMCwgMCwgMCwgMCwgMCwgMV0sIHUndXXigKY=


Output()

In [103]:
#Call this cell after you started the thread if you want to save CPU
stop_controller()

UR5WithGripperController emulation thread not started


In [104]:
#Call this cell to start the controller thread (the dot in the upper right of the notebook should flicker)
start_controller()
update_vis_from_controller()

Stopping UR5WithGripperController emulation thread...
UR5GripperAPI thread terminating
Stopped.
Starting UR5WithGripperController emulation thread...
UR5GripperAPI thread #1 starting


In [105]:
#test control api
wait_controller(1)
q = robot_control_api.getConfig()
q[1] += 1.5
q[6] = 0
print "Setting first config"
robot_control_api.setConfig(q)
wait_controller(2)
q[1] -= 1.5
q[6] = 1
print "Setting second config"
robot_control_api.setConfig(q)
print "Waiting for 2 seconds"
wait_controller(2)
print "Done with movement"

Setting first config
Setting second config
Waiting for 2 seconds
Done with movement


In [106]:
#load camera and block marker poses from vision
#set up world to reflect sensor data


#IDs of objects in marker_poses.txt file
object_ids = [1,2,3]
id_to_object = dict((id,i) for (i,id) in enumerate(object_ids))
#known transforms from marker coordinates to object coordinates
marker_object_transforms = [(so3.identity(),(0,0,0.025))]*3

#load the calibrated camera transform
Tcamera = loader.load('RigidTransform','calibrated_camera_xform.txt')
print Tcamera

kvis.add_xform("CAMERA")
kvis.set_transform("CAMERA",*Tcamera)

#load the current markers from marker_poses.txt
def update_objects_from_sensors():
    object_poses = []
    with open('marker_poses.txt','r') as datafile:
        lines = datafile.readlines()
        for line in lines:
            if len(line.strip())==0:
                continue
            id,xform = line.split(None,1)
            T = loader.read('RigidTransform',xform)
            #this transpose is because the transform was written in column-major format, but loader assumed row-major
            T = (so3.inv(T[0]),T[1])
            object_poses.append((int(id),T))
    #print "Read object poses",object_poses
    #reset objects to be far away if they are not
    for i in xrange(world.numRigidObjects()):
        world.rigidObject(i).setTransform(*se3.identity())
    for (id,T) in object_poses:
        if id not in id_to_object:
            raise RuntimeError("Invalid object ID specified, must be one of "+str(object_ids))
        ind = id_to_object[id]
        Tobj_c = se3.mul(T,se3.inv(marker_object_transforms[ind]))
        Tobj = se3.mul(Tcamera,Tobj_c)
        world.rigidObject(ind).setTransform(*Tobj)

#emulate camera sensor readings by writing to marker_poses.txt
def emulate_camera_sensor():
    global world,marker_object_transforms,object_ids,Tcamera
    with open('marker_poses.txt','w') as datafile:
        for i in xrange(controllerWorld.numRigidObjects()):
            Tm_c = se3.mul(se3.inv(Tcamera),se3.mul(world.rigidObject(i).getTransform(),marker_object_transforms[i]))
            datafile.write(str(object_ids[i]) + '    ' + loader.write((so3.inv(Tm_c[0]),Tm_c[1]),'RigidTransform')+'\n')


([-0.0217268228463, -0.990605306817, 0.135015077952, -0.851489884476, -0.0524355829894, -0.521742739548, 0.523920720914, -0.126299785201, -0.842351139641], [0.105938110847, 0.114881052998, 1.07113035801])


In [107]:
from klampt.model import ik
from klampt.model import collide
from klampt.plan import robotplanning

#information about the end effector
ee_link = 7
#the driver responsible for changing the gripper value
ee_driver = 6
#the position of the center link
ee_localpos = (0.15,0,0)
#the direction pointing from the palm toward the fingers
ee_fwd_dir = (1,0,0)
#the direction pointing perpendicular to the fingers
ee_perpendicular_dir = (0,0,1)

#driver parameters
ee_driver_open_value = 0.0
ee_driver_closed_value = 0.723

def set_gripper_width(robot,q,width):
    """Helper: produces a configuration which is the same as q, except that the
    gripper is opened to the given width (in the range 0 (closed) to 1 (open))"""
    qold = robot.getConfig()
    robot.setConfig(q)
    robot.driver(ee_driver).setValue(ee_driver_open_value*width + (1.0-width)*ee_driver_closed_value)
    qres = robot.getConfig()
    robot.setConfig(qold)
    return qres

def feasible(robot,q):
    """TODO: complete me, if it helps
    
    In:
    - robot: a RobotModel instance
    - q: a configuration
    
    Out: True if q is feasible (collision free, in joint limits)
    """
    qorig = robot.getConfig()
    qmin,qmax = robot.getJointLimits()
    
    for i in range(len(q)):
        if q[i] < qmin[i] or q[i] > qmax[i]:
            print "Joint"
            return False
    
    robot.setConfig(q)
    bool = robot.selfCollides()
    if bool is True:
            print "!!!!!!!!!!!!!!!!! Self collision !!!!!!!!!!!!!!!!!"
    robot.setConfig(qorig)
    return not bool

def visible(robot,qstart,qend):
    """TODO: complete me, if it helps
    
    In:
    - robot: a RobotModel instance
    - qstart,qend: start and end configurations
    
    Out: True if the straight line joint space path between qstart / qend is feasible
    """
    diff = vectorops.sub(qend, qstart)
    for i in range(100):
        # ignores first iteration because the very original configuration surpasses joint limits 
        if i>0:
            qnext = vectorops.madd(qstart, diff, i*1e-2) 
            if feasible(robot, qnext) is False:
                return False           
    return True

def closest_ik_solution(robot,q,qcurrent):
    """TODO: complete me, if it helps
    
    In:
    - robot: a RobotModel instance
    - q: a candidate IK solution, which has joint angles in the range [-2pi,2pi]
    - qcurrent: the current configuration
    
    Out: a modified version of q that has the same joint angles modulo 2pi (hence, has the
    same layout in workspace), but is closer to qcurrent so that motion is minimized.
    
    what if q1 = -3/4 pi but q2 is 2pi 
    q2 should become 0pi as it is closer
    
    what if q1= -1.5 pi and q2 = 0 pi
    q2 should become -2pi
    """
    
    alternative_q = q
    for i in range(len(q)):
        if q[i] <=0:
            alternative_q[i] = 2*pi + q[i]
            if abs(alternative_q[i]-qcurrent[i]) < abs(q[i]-qcurrent[i]) and visible(robot,qcurrent,alternative_q):
                q[i] = alternative_q[i]
            else: 
                alternative_q[i] = q[i]
        elif q[i] >=0:
            alternative_q[i] = -2*pi + q[i]
            if abs(alternative_q[i]-qcurrent[i]) < abs(q[i]-qcurrent[i]) and visible(robot,qcurrent,alternative_q):
                q[i] = alternative_q[i]
            else:
                alternative_q[i] = q[i]
        else:
            q[i] = q[i]
            
    return q

def move_home(robot):
    """
    TODO: complete me
    
    In:
    - robot: a RobotModel instance

    Out: a list of milestones interpolating from the robot's current configuration to the home configuration.
      
      None can be returned if your planner cannot find a solution.
    """  
    global qhome
    qstart = robot.getConfig()
    qcurrent = robot.getConfig()
    
    path = []
    path.append(qstart)
    new_path = find_path(robot,qstart,qhome)
    if new_path == None:
        return None
    path.extend(new_path)
    return path
                                    
def move_pregrasp(robot,block_pose,offset_height=0.06):
    """
    TODO: complete me
    
    In:
    - robot: a RobotModel instance, with the robot currently posed at qhome
    - block_pose: a transform of the center of the block
    - offset_height: the distance to hover above the block
    
    Out: a list of milestones interpolating from the robot's current configuration to the target pregrasp.
      
      None can be returned if your planner cannot find a solution.
    """
    qstart = robot.getConfig()
    qcurrent = robot.getConfig()
    
    block_location = block_pose[1]
    updated_block = (block_location[0],block_location[1],block_location[2]+offset_height)
    q_target = lab2b(robot,qhome,ee_link,ee_localpos,ee_fwd_dir,updated_block,(0,0,-1))[0]
    qend = set_gripper_width(robot,robot.getConfig(),1)
    robot.setConfig(qstart)
   
    path = []
    path.append(qstart)
    new_path = find_path(robot,qstart,qend)
    if new_path == None:
        return None
    path.extend(new_path)
    return path



def lower_and_close(robot,block_pose,close_width):
    """
    TODO: complete me
    
    In:
    - robot: a RobotModel instance, with the robot currently posed at the pregrasp configuration
    - block_pose: a transform of the center of the block
    - close_width: the amount that the gripper should be closed
    
    Out: a list of milestones interpolating from the robot's current configuration
    moving straight down (in Cartesian space), and then closing the robot's gripper.
    
    None can be returned if your planner cannot find a solution.
    """
    qstart = robot.getConfig()
    qdown = qstart[:]
    qcurrent = qstart
  
    path = []
    offset = 0.06
    path.append(qstart)
    q = None
    for i in range(6):
        offset = offset - 0.01
        block_location = block_pose[1]
        updated_block = (block_location[0],block_location[1],block_location[2]+offset)
        q = lab2b(robot,qhome,ee_link,ee_localpos,ee_fwd_dir,updated_block,(0,0,-1))[0]
        path_to_q = find_path(robot,qcurrent,q)
        qcurrent = q
        if path_to_q == None:
            return None
        path.extend(path_to_q)
    path.append(set_gripper_width(robot,qcurrent,close_width))
    return path
    
def lift_and_place(robot,target_position):
    """
    TODO: complete me
    
    In:
    - robot: a RobotModel instance, with the robot currently gripping an object
    - target_position: the position of the target box
    
    Out: a list of milestones interpolating from the robot's current configuration
    moving 1) to lift the block up, 2) to the target position, and 3) opening the gripper
    
    None can be returned if your planner cannot find a solution.
    """
    path = []
    qstart = robot.getConfig()
    path.append(qstart)
    #loc = robot.getLocation()
    end_link = robot.link(ee_link)
    current_location = end_link.getWorldPosition(ee_localpos)
    # GO UP
    new_location = (current_location[0],current_location[1],current_location[2]+0.1)
    qup = lab2b(robot,qhome,ee_link,ee_localpos,ee_fwd_dir,new_location,(0,0,-1))[0]
    path_to_qup = find_path(robot,qstart,qup)
    if path_to_qup == None:
        return None
    path.extend(path_to_qup)
    # GO OVER BOX
    qbox = lab2b(robot,qhome,ee_link,ee_localpos,ee_fwd_dir,target_position,(0,0,-1))[0]
    path_to_qbox = find_path(robot,qup,qbox)
    if path_to_qbox == None:
        return None
    path.extend(path_to_qbox)
    # OPEN SEASAME
    qopen = set_gripper_width(robot,qbox,1)
    path_to_qopen = find_path(robot,qbox,qopen)
    if path_to_qopen == None:
        return None
    path.extend(path_to_qopen)
    return path

def lab2b(robot,qseed,ee_link,ee_local_position,ee_local_axis,target,target_axis):
    """
    TODO: complete me
    
    In:
    - robot: a RobotModel instance
    - qseed: a seed configuration of the robot.
    - ee_link: the index of the end effector's link
    - ee_local_position: the position of the end effector, expressed in the frame
      of link ee_link.
    - ee_local_axis: the rotational axis of the end effector, expressed in
      the frame of link ee_link.
    - target: a 3d target position (x,y,z)
    - target_axis: a 3d target direction for the end effector's axis (x,y,z)
    Out: a triple (q,errpos,erraxis) giving
    - q: the inverse kinematics optimized configuration of the robot, which (locally)
      minimizes the distance between the end effector position/axis and the target position/axis
    - errpos: The final distance between the end effector's world position
      and the target position. (should be as close as possible to 0 given a
      successful solution)
    - erraxis: The final distance between the end effector's world axis and
      the target axis.
    You will need to examine the documentation of the klampt.ik module, specifically
    the ik.objective and ik.solve functions.
    """
    #TODO: put your code here
    ##HINT: set up an IK objective as follows to strictly constrain the link's
    ##position and orientation
    #goal = ik.objective(robot.link(ee_link),R=desired_rotation_of_link,t=desired_translation_of_link)
    ##OR if you want to match some points lp1,lp2,..., given in the local
    ##coordinate frame to points wp1,wp2,..., given in the global coordinate
    ##frame
    target_mag = math.sqrt(target_axis[0]**2+target_axis[1]**2+target_axis[2]**2)
    target_unit = (target_axis[0]/target_mag,target_axis[1]/target_mag,target_axis[2]/target_mag)
    
    local_mag = math.sqrt(ee_local_axis[0]**2+ee_local_axis[1]**2+ee_local_axis[2]**2)
    local_unit = (ee_local_axis[0]/target_mag,ee_local_axis[1]/target_mag,ee_local_axis[2]/target_mag)
    
   
    
    local_plus_unit = vectorops.add(local_unit,ee_local_position)
    target_plus_unit = vectorops.add(target_unit,target)
    goal = ik.objective(robot.link(ee_link),local=[ee_local_position,local_plus_unit],world=[target,target_plus_unit])
    ##now seed the ik solver with the correct configuration (READ THE DOCS)...
    #???
    ##now solve
    if ik.solve(goal):
        print "IK success!"
    ##   now the robot model's configuration is changed, and you need to
    ##   extract it. Your errpos and erraxis values should be very small
    else:
        print "IK failure... returning best solution found"
    ##   the robot model's configuration is changed, but your errpos and
    ##   erraxis will not be close to 0
    worldPos = robot.link(ee_link).getWorldPosition(ee_local_position)
    worldAxis = robot.link(ee_link).getWorldDirection(ee_local_axis)
    ee_world_position = ee_local_position
    ee_world_direction = ee_local_axis
    return (robot.getConfig(),vectorops.distance(worldPos,target),vectorops.distance(worldAxis,target_axis))

def find_path(robot,qstart,qend):
    qcurrent = qstart
    print visible(robot, qstart, qend)
    if visible(robot, qstart, qend) is True:
        return [qend]
    #sampler
    else:
        path = []
        #path.append(qstart)
        for j in range(100): 
            random_q = random_ik_seed(robot)
            print random_q
            print "*************Checking if random is visible from current************"
            if visible(robot, qcurrent, random_q):
                print "------Random visible from current------------"
                random_q = closest_ik_solution(robot,random_q,qcurrent)
                path.append(random_q)
                qcurrent = random_q
            print "*************Checking if new config to target works************"
            if visible(robot, qcurrent, qend) is True:
                print "------target visible from current------------"
                qend = closest_ik_solution(robot,qend,qcurrent)
                path.append(qend)
                return path
    return None

def random_ik_seed(robot):
    """Returns an IK seed for use in the local IK solver"""
    q = [0.0]*robot.numLinks()
    qmin,qmax = robot.getJointLimits()
    for i in range(ee_link):
        q[i] = random.uniform(qmin[i],qmax[i])
    return q


In [108]:
#run this cell to bind your code to the visualization playback window. You should 
#not need to modify this code, unless you want to tweak the trajectory execution speed

#target parameters
target_position = (0.26,-0.15,1.2)

#block parameters
block_width = 0.05
block_names = [world.rigidObject(i).getName() for i in range(world.numRigidObjects())]
block_poses = [world.rigidObject(i).getTransform() for i in range(world.numRigidObjects())]

#gripper parameters
block_close_amount = 0.5

#the speed of trajectory execution: default 0.5 radian per second
#this may be tweaked, if desired
trajectory_execution_speed = 0.5

from klampt.model.trajectory import RobotTrajectory

#state can be 'HOME', 'PREGRASP', 'GRASPED', 'BOX'
state = 'HOME'

kvis.add_text("HUD1",1,1)
kvis.add_text("HUD2",1,5)
kvis.add_text("HUD3",1,9)
kvis.add_text("HUD4",1,13)
kvis.add_text("HUD5",1,17)
kvis.add_text("HUD6",1,21)
kvis.add_text("HUD7",1,25)

#the amount of time between refreshes
dt = 0.05
                
def execute_trajectory(traj):
    """Executes a trajectory on the controller.  You may or may not want to modify this."""
    global robot,controls
    for c in controls.children:
        c.disabled = True
        
    #send the trajectory
    dtraj = traj.discretize(dt)
    q0 = robot_control_api.getConfig()
    q0_traj = klampt_to_controller(traj.milestones[0])
    robot_control_api.setConfig(klampt_to_controller(traj.milestones[0]))
    t0 = traj.times[0]
    for t,q in zip(dtraj.times[1:],dtraj.milestones[1:]):
        wait_controller(t-t0)
        robot_control_api.setConfig(klampt_to_controller(q))
        qold = robot.getConfig()
        robot.setConfig(q)
        if robot.selfCollides():
            print "~~~~~~~~~~~~~~~Self collision in trajectory~~~~~~~~~~~~~~~~"
        robot.setConfig(qold)
        t0 = t
    
    for c in controls.children:
        c.disabled = False

def trajectory_from_path(path):
    """TODO: perform a smoother path interpolation"""
    global robot
    times = [0.0]
    for i in range(1,len(path)):
        d = robot.distance(path[i-1],path[i])
        times.append(times[-1]+d/trajectory_execution_speed)
    print "Inferred path duration",times[-1]-times[0]
    return RobotTrajectory(robot,times,path)

def execute_path(path):
    execute_trajectory(trajectory_from_path(path))
    
def do_move_home(button):
    global state,robot,qhome
    output_area.clear_output()
    with output_area:
        try:
            
            milestones = move_home(robot)
            if milestones is not None:
                execute_path(milestones)
        except Exception:
            import traceback
            traceback.print_exc()
    state = 'HOME'

def do_sense(button):
    output_area.clear_output()
    with output_area:
        if CONTROLLER == 'simulation':
            emulate_camera_sensor()
        update_objects_from_sensors()
    kvis.update()
    
def do_move_pregrasp(button):
    global state,robot
    output_area.clear_output()
    with output_area:
        try:
            if state != 'HOME':
                print "WARNING: should do pregrasp from the home configuration"
            robot.setConfig(controller_to_klampt(robot_control_api.getConfig()))
            selected_object = object_dropdown.value
            milestones = move_pregrasp(robot,world.rigidObject(selected_object).getTransform())
            if milestones is not None:
                execute_path(milestones)
                state = 'PREGRASP'
        except Exception:
            import traceback
            traceback.print_exc()
        
            
def do_grasp(button):
    global state,robot
    output_area.clear_output()
    with output_area:
        try:
            if state != 'PREGRASP':
                print "WARNING: should do grasp from the pregrasp configuration"
            selected_object = object_dropdown.value
            robot.setConfig(controller_to_klampt(robot_control_api.getConfig()))
            milestones = lower_and_close(robot,world.rigidObject(selected_object).getTransform(),block_close_amount)
            if milestones is not None:
                execute_path(milestones)
                state = 'GRASPED'
        except Exception:
            import traceback
            traceback.print_exc()

def do_place(button):
    global state,robot
    output_area.clear_output()
    with output_area:
        try:
            if state != 'GRASPED':
                print "WARNING: should do place from the grasped configuration"
            robot.setConfig(controller_to_klampt(robot_control_api.getConfig()))
            milestones = lift_and_place(robot,target_position)
            if milestones is not None:
                execute_path(milestones)
                state = 'BOX'
        except Exception:
            import traceback
            traceback.print_exc()
        
object_dropdown = widgets.Dropdown(
    options=block_names,
    value=block_names[0],
    description='Object:'
)

move_home_button = widgets.Button(description="Move Home")
sense_button = widgets.Button(description="Sense")
move_pregrasp_button = widgets.Button(description="Move to Pregrasp")
grasp_button = widgets.Button(description="Grasp")
place_button = widgets.Button(description="Place")
move_home_button.on_click(do_move_home)
sense_button.on_click(do_sense)
move_pregrasp_button.on_click(do_move_pregrasp)
grasp_button.on_click(do_grasp)
place_button.on_click(do_place)

controls.children = (move_home_button,sense_button,object_dropdown,move_pregrasp_button,grasp_button,place_button)

## Written answers

Problem B

Problem C